In [ ]:
from __future__ import print_function

import numpy as np
from matplotlib import pyplot as plt
import GPy
from scipy.stats import norm
from scipy.optimize import minimize
np.int = int # проблема в устаревшей библиотеке
np.bool = bool

`
pip3 install Gpy
pip3 install ipywidgets
`

in case of python 3.7 error while installing


`git clone https://github.com/SheffieldML/GPy
find Gpy -name '*.pyx' -exec cython {} \;
pip3 install Gpy/`

https://github.com/SheffieldML/GPy/issues/649

In [ ]:
from ipywidgets import interact, interactive, widgets, fixed
try:
    from ipywidgets import Layout
except:
    pass

## функции для оптимизации и отображения

In [ ]:
def log_expected_improvement(mean_values, variance_values, opt_value):
    estimated_values = mean_values.ravel()
    eps = 0.05 / len(estimated_values)

    delta = (opt_value - estimated_values - eps).ravel()

    estimated_errors = (variance_values ** 0.5).ravel()

    non_zero_error_inds = np.where(estimated_errors > 1e-6)[0]
    Z = np.zeros(len(delta))
    Z[non_zero_error_inds] = delta[non_zero_error_inds] / estimated_errors[non_zero_error_inds]
    log_EI = np.log(estimated_errors) + norm.logpdf(Z) + np.log(1 + Z * np.exp(norm.logcdf(Z) - norm.logpdf(Z)))
    return log_EI


def get_new_point(model, lb, ub, data=None, multistart=50, random_state=None):
    """
    Parameters:
        model - GP model of the objective function
        lb, ub - array-like, lower and upper bounds of x
        data - tuple(x_train, y_train)
        multistart - number of multistart runs
        random_state - np.random.RandomState
    Returns
        tuple - argmin of the objective function and min value of the objective
    """
    if random_state is None:
        random_state = np.random.RandomState()

    lb = np.array(lb).reshape(1, -1)
    ub = np.array(ub).reshape(1, -1)

    # 1. Generate inital X points (number of points == multistart) in [lb, ub]

    x_random = random_state.uniform(size=(multistart, np.array(lb).ravel().shape[0]))
    x_random *= ub - lb
    x_random += lb

    def objective(x):
        if x.ndim == 1:
            x = x.reshape(1, -1)
        mean_values, variance = model.predict(x)
        std_values = np.sqrt(variance)
        return -log_expected_improvement(mean_values, std_values, data[1].min())

    criterion_value = objective(x_random)

    # 2. From each points from x_random run L-BFGS optimization algorithm,
    #    choose the best result and return it
    #    Use function minimize: minimize(objective, x_init, method='L-BFGS-B',
    #                                    bounds=np.vstack((lb, ub)).T)
    #    it returns object with fields 'fun' - optimum function value, 'x' - argmin.

    best_result = None
    best_value = np.inf

    for x_init in x_random:
        optimization_result = minimize(objective, x_init, method='L-BFGS-B', bounds=np.vstack((lb, ub)).T)

        if optimization_result.fun < best_value:
            best_result = optimization_result
            best_value = best_result.fun[0]
    return best_result.x, best_result.fun


def get_model_values(model, x, y_train):
    prediction, variance = model.predict(x)
    std = np.sqrt(variance).ravel()

    log_EI = np.exp(log_expected_improvement(prediction, std, y_train.min()))

    values = [prediction, log_EI, std]
    return values

In [ ]:
def _plot_2D(x_grid, values, x_new, x_train, need_log=True):
    """subprogram for plotting 2D values"""
    # узнаём длину ребра = корень из "площади"
    grid_size = int(np.sqrt(len(x_grid[:, 0])))
    # строим grid в нужной нотации (с продольной на квадратную)
    tX_grid = x_grid[:, 0].reshape(grid_size, grid_size)
    tY_grid = x_grid[:, 1].reshape(grid_size, grid_size)
    # меняем нотацию матрицы (-//-)
    Y_matrix = values.reshape(grid_size, grid_size)
    # поскольку мы работаем с маленькими значениями параметров, переведём их в "научный" формат
    plt.ticklabel_format(axis='y', style='sci', scilimits=(0, 0))
    plt.ticklabel_format(axis='x', style='sci', scilimits=(0, 0))
    # строим само распределение. В логорифмическом масштабе или без оного
    if need_log:
        plt.pcolormesh(tX_grid, tY_grid, Y_matrix, norm=LogNorm())
    else:
        plt.pcolormesh(tX_grid, tY_grid, Y_matrix)
    plt.colorbar()
    # обозначим, где находится "0" -- без добавок
    plt.axhline(0, color="w", linewidth=0.8, linestyle=":")
    plt.axvline(0, color="w", linewidth=0.8, linestyle=":")

    # обозначим точки, где уже знаем функцию
    plt.scatter(x_train[:, 0], x_train[:, 1], c='r', s=20)
    # и если уже решили, где будем искать следующее значение -- обозначим место тоже
    if x_new is not None:
        plt.axvline(x_new[0], color="green")
        plt.axhline(x_new[1], color="green")


def _plot_1D(x_grid, values, x_new, x_train, y_train, train_display, need_log=True):
    """subprogram for plotting 1D values"""
    # поскольку мы работаем с маленькими значениями параметров, переведём их в "научный" формат
    plt.ticklabel_format(axis='x', style='sci', scilimits=(0, 0))
    # отображаем тренировочный сет. Для y_predicted и EI отображение отличается, поэтому выносим его
    # в отдельную (так же передаваемую) функцию
    train_display(x_train, y_train)
    # отображем распределение
    plt.plot(x_grid, values.ravel(), '-k', linewidth=2)
    # показываем, где 0
    plt.axvline(0, linewidth=0.8, linestyle=":")
    # если хотим логорифмический масштаб,
    if need_log:
        plt.yscale("log")
    if x_new is not None:
        plt.axvline(x_new, color="green")


def plot_model_EI(x_train, y_train, x_grid, y_predict, std_, log_EI=None, x_new=None):
    # смотрим размерность пространства (1D или 2D)
    is_dim1 = x_train.shape[1] == 1
    is_dim2 = x_train.shape[1] == 2
    # один или два графика? Если два, фигура побольше и подграфики делаем
    if log_EI is not None:
        plt.figure(figsize=(16, 6))
        plt.subplot(121)
    else:
        plt.figure(figsize=(8, 6))

    # рисуем предсказания модели
    plt.title("y predict")
    # для 1D
    if is_dim1:
        _plot_1D(
            x_grid, y_predict, x_new, x_train, y_train,
            # тренировочный сет обозначем маркерами
            train_display=lambda xs, ys: plt.plot(xs, ys, 'or', markersize=8),
            need_log=False
        )
        # плюсом добавим, насколько мы не уверены в результате?
        prediction = y_predict.ravel()
        std = std_.ravel()
        plt.fill_between(x_grid.ravel(), prediction - 2 * std, prediction + 2 * std, alpha=0.3)
    # для 2D
    if is_dim2:
        _plot_2D(x_grid, y_predict, x_new, x_train, need_log=False)

    # рисуем log EI
    if log_EI is not None:
        plt.subplot(122)
        plt.title("log EI")
        # для 1D
        if is_dim1:
            _plot_1D(
                x_grid, log_EI, x_new, x_train, y_train=None,
                # тренировочный сет обозначаем вертикальными линиями
                train_display=lambda xs, _: np.vectorize(
                    lambda x: plt.axvline(x, color="r", linewidth=0.8, linestyle=":")
                )(xs),
                need_log=False
            )
        # для 2D
        if is_dim2:
            _plot_2D(x_grid, log_EI, x_new, x_train, need_log=False)
    plt.show()

In [ ]:
def oracul(x):
    return float(input("введите значение в точке {}: ".format(x)))

## оптимизация для одного параметра

### границы

In [ ]:
lower_bounds = [0]
upper_bounds = [1]

### тренировочный набор

In [ ]:
x_train = np.array([0, 0.2, 0.8, 1.0])

(значение функции возьмите из файла `04_function`)

In [ ]:
y_train = np.array([oracul(x) for x in x_train])

In [ ]:
x_train = x_train.reshape(-1, 1)
y_train = y_train.reshape(-1, 1)

print(x_train)
print(y_train)

### посмотрим, что имеем в начале

In [ ]:
kernel = GPy.kern.RBF(1)
model = GPy.models.GPRegression(x_train, y_train, kernel)
model.Gaussian_noise.variance = 0.00001
model.Gaussian_noise.variance.fix()
model.optimize_restarts(verbose=False, num_restarts=10)

grid_size = 50
x_grid = np.linspace(lower_bounds[0], upper_bounds[0], grid_size).reshape(-1,1)

In [ ]:
y_predict, log_EI, std = get_model_values(model, x_grid, y_train)
plot_model_EI(x_train, y_train, x_grid, y_predict, std, log_EI, x_new=None)

### оптимизация

In [ ]:
for i in range(5):
    kernel = GPy.kern.RBF(1)
    model = GPy.models.GPRegression(x_train, y_train, kernel)
    model.Gaussian_noise.variance = 0.00001
    model.Gaussian_noise.variance.fix()
    model.optimize_restarts(verbose=False, num_restarts=10)

    grid_size = 50
    x_grid = np.linspace(lower_bounds[0], upper_bounds[0], grid_size).reshape(-1,1)
    x_new, criterion_value = get_new_point(
        model, data=(x_train, y_train), lb=lower_bounds, ub=upper_bounds
    )
    
    y_predict, log_EI, std = get_model_values(model, x_grid, y_train)
    plot_model_EI(x_train, y_train, x_grid, y_predict, std, log_EI, x_new)
    x_new = x_new.reshape(1, -1)
    new_value = np.array([oracul(x_new.ravel()[0])]).reshape(1, -1)
    print(x_new, "-->", new_value)
    x_train = np.vstack([x_train, x_new])
    y_train = np.vstack([y_train, new_value])

### и в итоге получаем:

In [ ]:
kernel = GPy.kern.RBF(1)
model = GPy.models.GPRegression(x_train, y_train, kernel)
model.Gaussian_noise.variance = 0.00001
model.Gaussian_noise.variance.fix()
model.optimize_restarts(verbose=False, num_restarts=10)

grid_size = 50
x_grid = np.linspace(lower_bounds[0], upper_bounds[0], grid_size).reshape(-1,1)
y_predict, log_EI, std = get_model_values(model, x_grid, y_train)
plot_model_EI(x_train, y_train, x_grid, y_predict, std, log_EI=None, x_new=None)

## попробуем для 2D

In [ ]:
lower_bounds = [0, -1]
upper_bounds = [1, 1]

In [ ]:
def oracul2D(X):
    r = np.linalg.norm(X)
    return r**5 + r + np.sin(4*np.pi * r) - 5*(1/np.sqrt((X[0]-1)**2+1)) - 3*np.abs(X[1])


In [ ]:
x_train = np.array([
    [0, 0],
    [1, 1],
    [0.8, 0.2],
    [0.3, 0.75], 
    [1, -1]
])

In [ ]:
y_train = np.array([oracul2D(x) for x in x_train]).reshape(-1,1)

In [ ]:
grid_size = 50
xy_grid = np.meshgrid(
    np.linspace(lower_bounds[0], upper_bounds[0], grid_size), 
    np.linspace(lower_bounds[1], upper_bounds[1], grid_size)
)
x_grid = np.hstack((xy_grid[0].reshape(-1, 1), xy_grid[1].reshape(-1, 1)))

In [ ]:
kernel = GPy.kern.RBF(2, ARD=False)  # ARD=False значит, что характерная длина по x и y совпадают
model = GPy.models.GPRegression(x_train, y_train, kernel)
model.Gaussian_noise.variance = 0.00001
model.Gaussian_noise.variance.fix()
model.optimize_restarts(verbose=False, num_restarts=10);

y_predict, log_EI, std = get_model_values(model, x_grid, y_train)
plot_model_EI(x_train, y_train, x_grid, y_predict, std, log_EI, x_new=None)

In [ ]:
for i in range(15):
    kernel = GPy.kern.RBF(2, ARD=False)
    model = GPy.models.GPRegression(x_train, y_train, kernel)
    model.Gaussian_noise.variance = 0.00001
    model.Gaussian_noise.variance.fix()
    model.optimize_restarts(verbose=False, num_restarts=10)

    x_new, criterion_value = get_new_point(
        model, data=(x_train, y_train), lb=lower_bounds, ub=upper_bounds
    )
    
    y_predict, log_EI, std = get_model_values(model, x_grid, y_train)
    plot_model_EI(x_train, y_train, x_grid, y_predict, std, log_EI, x_new)
    x_new = x_new.reshape(1, -1)
    new_value = np.array([oracul2D(x_new.ravel())]).reshape(1, -1)
    print(x_new, "-->", new_value)
    x_train = np.vstack([x_train, x_new])
    y_train = np.vstack([y_train, new_value])

In [ ]:
y_predict, log_EI, std = get_model_values(model, x_grid, y_train)
plot_model_EI(x_train, y_train, x_grid, y_predict, std, log_EI=None, x_new=None)

## а как же эта функция выглядит на самом деле?

In [ ]:
y_predict = np.array([oracul2D(x) for x in x_grid]).reshape(-1,1)
plot_model_EI(x_train, y_train, x_grid, y_predict, std, log_EI=None, x_new=None)

### Где у неё минимум?

In [ ]:
y_predict.min(), x_grid[y_predict.argmin()]

### как видите, минимум в нижней половине. Мы её вообще не смотрели! 

## Вывод -- даже с этим алгоритмом надо быть аккуратным и не полностью полагаться на его результаты